## centroid(h2)

In [1]:
import geopandas as gpd
import pandas as pd

# Access built-in Natural Earth data via GeoPandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Get a list (dataframe) of country centroids
centroids = world.centroid
centroids = pd.concat([world.name, centroids], axis=1)

centroids.rename(columns={0:'centroid'},inplace=True)  

In [2]:
#extract long ,lat from centroid point data type
centroids['long']=centroids['centroid'].apply(lambda x: x.x)
centroids['lat']=centroids['centroid'].apply(lambda x: x.y)
centroids.head()

,name,centroid,long,lat
0,Fiji,POINT (163.85316 -17.31631),163.853165,-17.316309
1,Tanzania,POINT (34.75299 -6.25773),34.752990,-6.257732
2,W. Sahara,POINT (-12.13783 24.29117),-12.137831,24.291173
3,Canada,POINT (-98.14238 61.46908),-98.142381,61.469076
4,United States of America,POINT (-112.59944 45.70563),-112.599436,45.705628


In [3]:
centroids[centroids['name']=='China']

,name,centroid,long,lat
139,China,POINT (103.88361 36.55507),103.883612,36.555067


In [4]:
import country_converter as coco

centroids['iso3']=centroids.name.apply(lambda x: coco.convert(names=x,to='ISO3') )

In [31]:
centroids.loc[centroids['iso3']=='FRA','lat']=48
centroids.loc[centroids['iso3']=='FRA','long']=5

In [32]:
centroids[centroids['iso3']=='FRA']

,Unnamed: 0,name,centroid,long,lat,iso3
43,43,France,POINT (-2.876696699270627 42.46070432663372),5.0,48.0,FRA


In [33]:
centroids.to_csv('./centroids.csv')

## flow_betw_country(h2)

In [16]:
import pandas as pd

In [17]:
ques=pd.read_csv('data16-19NoText/stackoverflow_questions_2016-2019_notext.csv')[['Id','owner_user_id']]
user=pd.read_csv('data16-19NoText/stackoverflow_users_all_notext_clean.csv')[['id','country_iso3']]
ans=pd.read_csv('data16-19NoText/stackoverflow_answers_2016-2019_notext.csv')[['Id','owner_user_id','parent_id']]

In [34]:
centroids=pd.read_csv('./centroids.csv')

In [35]:
# merge ans ques into one table
ans_ques=pd.merge(ans,ques,left_on='parent_id',right_on='Id',suffixes=('_ans', '_ques'),)

# ans=ans.rename(columns={'owner_user_id':'ans_owner_id','parent_id':'ques_owner_id'})
# ans.head()

In [36]:
ans_ques.head()

,Id_ans,owner_user_id_ans,parent_id,Id_ques,owner_user_id_ques
0,56292528,5674777.0,56275396,56275396,5674777.0
1,56292773,8245406.0,56292148,56292148,11430866.0
2,56292918,8506548.0,56292148,56292148,11430866.0
3,56293594,8380272.0,56292148,56292148,11430866.0
4,56292786,8199990.0,49432666,49432666,6683866.0


In [37]:
#merge ans id column with user table to generate ans_owner_country
ans_ques=pd.merge(ans_ques,user,left_on='owner_user_id_ans',right_on='id',copy=False,how='inner')\
.rename(columns={'country_iso3':'ans_owner_country'})\
.drop('id',axis=1)
#merge ques id column with user table to generate ques_owner_country
ans_ques=pd.merge(ans_ques,user,left_on='owner_user_id_ques',right_on='id',copy=False,how='inner')\
.rename(columns={'country_iso3':'ques_owner_country'})\
.drop('id',axis=1)
ans_ques.head()
#executed in 14.2s

,Id_ans,owner_user_id_ans,parent_id,Id_ques,owner_user_id_ques,ans_owner_country,ques_owner_country
0,56292528,5674777.0,56275396,56275396,5674777.0,NLD,NLD
1,42777028,5988389.0,42754732,42754732,5674777.0,THA,NLD
2,44013446,979387.0,44011323,44011323,5674777.0,USA,NLD
3,44012639,8022293.0,44011323,44011323,5674777.0,USA,NLD
4,56292918,8506548.0,56292148,56292148,11430866.0,DEU,IND


In [38]:
#group by both ques and ans country
country_flow=ans_ques.groupby(by=['ques_owner_country','ans_owner_country']).count()['Id_ans']
# convert hierachical index to columns
country_flow=country_flow.reset_index().sort_values('Id_ans',ascending=False).rename(columns={'Id_ans':'count'}).reset_index()

In [39]:
country_flow.head()

,index,ques_owner_country,ans_owner_country,count
0,16886,USA,USA,237222
1,7136,IND,IND,187438
2,7243,IND,USA,67089
3,16780,USA,IND,49883
4,5579,GBR,GBR,45615


In [40]:
country_flow=pd.merge(country_flow,centroids[['iso3','long','lat']],left_on='ques_owner_country',right_on='iso3')\
.drop(columns='iso3')

In [41]:
country_flow=pd.merge(country_flow,centroids[['iso3','long','lat']],left_on='ans_owner_country',right_on='iso3'\
                      ,suffixes=('_ques','_ans')).drop(columns='iso3')

In [42]:
country_flow

,index,ques_owner_country,ans_owner_country,count,long_ques,lat_ques,long_ans,lat_ans
0,16886,USA,USA,237222,-112.599436,45.705628,-112.599436,45.705628
1,7243,IND,USA,67089,79.593704,22.925006,-112.599436,45.705628
2,5699,GBR,USA,35947,-2.853135,53.914773,-112.599436,45.705628
3,4098,DEU,USA,22288,10.288485,51.133723,-112.599436,45.705628
4,2632,CAN,USA,22002,-98.142381,61.469076,-112.599436,45.705628
...,...,...,...,...,...,...,...,...
15105,5158,FIN,GAB,1,26.211765,64.504094,11.687751,-0.647048
15106,10008,MAR,GAB,1,-8.420480,29.885395,11.687751,-0.647048
15107,5513,GAB,GAB,8,11.687751,-0.647048,11.687751,-0.647048
15108,12930,PHL,NER,1,122.902672,11.763799,9.324427,17.345553


In [43]:
centroids[centroids['iso3']=='FRA']

,Unnamed: 0,Unnamed: 0.1,name,centroid,long,lat,iso3
43,43,43,France,POINT (-2.876696699270627 42.46070432663372),5.0,48.0,FRA


In [44]:
country_flow.to_csv('flow_betw_country.csv')

In [45]:
country_flow[country_flow['ans_owner_country']=='FRA']

,index,ques_owner_country,ans_owner_country,count,long_ques,lat_ques,long_ans,lat_ans
991,16755,USA,FRA,13779,-112.599436,45.705628,5.0,48.0
992,7114,IND,FRA,15114,79.593704,22.925006,5.0,48.0
993,5578,GBR,FRA,5770,-2.853135,53.914773,5.0,48.0
994,3983,DEU,FRA,5118,10.288485,51.133723,5.0,48.0
995,2520,CAN,FRA,2365,-98.142381,61.469076,5.0,48.0
...,...,...,...,...,...,...,...,...
1151,15464,TCD,FRA,2,18.581330,15.328867,5.0,48.0
1152,4732,ERI,FRA,1,38.678187,15.427277,5.0,48.0
1153,1152,BDI,FRA,1,29.913892,-3.377391,5.0,48.0
1154,5989,GIN,FRA,1,-11.060854,10.448273,5.0,48.0


In [46]:
country_flow[country_flow['ans_owner_country']=='DEU']

,index,ques_owner_country,ans_owner_country,count,long_ques,lat_ques,long_ans,lat_ans
508,16740,USA,DEU,29088,-112.599436,45.705628,10.288485,51.133723
509,7101,IND,DEU,28747,79.593704,22.925006,10.288485,51.133723
510,5567,GBR,DEU,11916,-2.853135,53.914773,10.288485,51.133723
511,3972,DEU,DEU,37638,10.288485,51.133723,10.288485,51.133723
512,2508,CAN,DEU,4779,-98.142381,61.469076,10.288485,51.133723
...,...,...,...,...,...,...,...,...
665,14903,SUR,DEU,4,-55.911456,4.120008,10.288485,51.133723
666,5292,FLK,DEU,1,-59.420973,-51.713222,10.288485,51.133723
667,11523,NER,DEU,1,9.324427,17.345553,10.288485,51.133723
668,9578,LSO,DEU,1,28.170105,-29.625290,10.288485,51.133723


In [48]:
!pwd

/Users/saibo/AEPFL_S2/Data Viz/milestone2/saibo/data
